In [24]:
# Download all packages 
import pickle
import detritalpy.detritalFuncs as dFunc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from random import choices
from random import randrange
from random import randint
import random
import time
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import MaxNLocator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from tpot import TPOTClassifier

In [25]:
# Select the dataset to train the TPOT Classifier on
# Here, we select the fixed age mode 0.95-0.99 similarity interval
cut_0 = pickle.load(open('0.95-0.99 - 0 Sources.p','rb'))
cut_1 = pickle.load(open('0.95-0.99 - 1 Sources.p','rb'))
cut_2 = pickle.load(open('0.95-0.99 - 2 Sources.p','rb'))
cut_3 = pickle.load(open('0.95-0.99 - 3 Sources.p','rb'))
cut_4 = pickle.load(open('0.95-0.99 - 4 Sources.p','rb'))
cut_5 = pickle.load(open('0.95-0.99 - 5 Sources.p','rb'))
cut_6 = pickle.load(open('0.95-0.99 - 6 Sources.p','rb'))
cut_7 = pickle.load(open('0.95-0.99 - 7 Sources.p','rb'))
init_data = {**cut_0,**cut_1,**cut_2,**cut_3,**cut_4,**cut_5,**cut_6,**cut_7}

# We need to select a subset of the data to run the TPOT classifier on
# Lets use 5 sources at 115 analyses

analyses_number = 115
source_number = 5
source_nums = np.random.choice((np.arange(0,8)),source_number,replace=False)

auto_data = {}
for key in init_data:
    if key[0] in source_nums and key[1] == analyses_number:
        auto_data[key] = init_data[key]

# Format data into a dataframe
ID_list = []
KDE_list = []
for entry in auto_data:
    source_ID = entry[0]
    for KDE in auto_data[entry]:
        KDE_list.append(KDE)
        ID_list.append(source_ID)
final_df = pd.DataFrame()
final_df['ID'] = ID_list
final_df['KDE'] = KDE_list

# Data preprocessing
train_set,test_set = train_test_split(final_df,test_size = 0.2, random_state = randrange(42))
train_KDE = list(train_set['KDE'])
train_labels = list(train_set['ID'])
test_KDE = list(test_set['KDE'])
test_labels = list(test_set['ID'])
enc = OrdinalEncoder()
train_labels = np.array(train_labels)
train_labels = train_labels.reshape(-1,1)
train_labels = enc.fit_transform(train_labels)
test_labels = np.array(test_labels)
test_labels = test_labels.reshape(-1,1)
test_labels = enc.fit_transform(test_labels)
train_KDE = np.array(train_KDE)
test_KDE = np.array(test_KDE)

In [22]:
# Now run the TPOT classifier
# Here, we specify the TPOT classifier hyperparamters
# These hyperparamters represent a low end case that prioritizes speed over optimization
# For better results, increase the generations and max_eval_time_mins

# Number of iterations to run the pipeline optimization process (default=100)
generations = 1
# How much information TPOT communicates (0-3)
verbosity = 2
# The seed of the pseudo random number generator used in TPOT
random_state = 42
# Number of processes to use in parallel for evaluating pipelines during the TPOT optimization process
n_jobs=-2
# How many minutes TPOT has to evaluate a single pipeline
max_eval_time_mins = 2
# Function used to evaluate the quality of a given pipeline for the classification problem
scoring = 'f1_weighted'

tpot = TPOTClassifier(generations=generations, verbosity=verbosity, random_state=random_state,n_jobs=n_jobs,max_eval_time_mins = max_eval_time_mins,scoring=scoring)

In [23]:
# Now we train and export the optimized pipeline
model = tpot.fit(train_KDE, train_labels.ravel())
print(tpot.score(test_KDE, test_labels.ravel()))
tpot.export('Optimized TPOT Pipeline Example.py')

Optimization Progress:   0%|          | 0/200 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.48583333333333323

Best pipeline: LogisticRegression(StandardScaler(input_matrix), C=15.0, dual=False, penalty=l2)
0.22857142857142856
